In [ ]:
# Credit to Jonathan Whitaker

In [ ]:
from PIL import Image
from scipy import spatial
import numpy as np
import random
from matplotlib import pyplot as plt

In [ ]:
from demo import get_train_data
dataset=get_train_data()
# from datasets import load_dataset
# dataset=load_dataset("image_folder",data_dir="./assets/outputs")

In [ ]:
# target_image = dataset['train'][4]['image'] # Using one of the dataset as a demo
target_image = Image.open("./assets/mosaic_bg.png") # <<< To use your own image
rgba = np.array(target_image)
rgba[rgba[...,-1]==0] = [255,255,255,0] # Make image transparent white anywhere it is transparent
target_image=Image.fromarray(rgba).convert("RGB")
target_image

In [ ]:
# Pixelate (resize) the target photo: This is done so that indexing is easier.

# Specify width and height directly if you don't care about keeping aspect ratio:
width, height = 31, 31

# Adjust them so the larger dimension is <size> to roughly follow aspect ratio: 
size = 32
# w, h = target_image.size
# if w>h:
#   width = int(size)
#   height = int(size*h/w)
# else:
#   width = int(size*w/h)
#   height = int(size)

print('Downscaling to', width, height)

required_num_images=(width*height)+1
print(f"we will need {required_num_images} number of images")

resized_target_image = target_image.resize((width, height))
resized_target_image

In [ ]:
# Load all tiles and get mean color
tile_size = (128, 128) # Set lower if the final result is too high-res for your liking

def resize(ex):
    ex["image"]=ex["image"].resize((tile_size[0],tile_size[1]))
    im_arr=np.array(ex["image"])
    mean_color = im_arr.mean(axis=0).mean(axis=0)
    # mean_color = np.array(ex["image"]).mean(axis=0).mean(axis=0)
    ex["mean_color"]=mean_color
    return ex

unused_cols = dataset.column_names
unused_cols.remove("image")
dataset = dataset.remove_columns(unused_cols)
dataset = dataset.map(resize) #resize before or OOM
dataset["mean_color"][0]

In [ ]:
def make_mosaic(dataset,shuffle=True, remove_when_used=True):
    # Create an output image
    output = Image.new('RGB', (width*tile_size[0], height*tile_size[1]))

    colors =dataset["mean_color"].copy()
    
    tiles = dataset["image"]
    # Create a KDTree
    tree = spatial.KDTree(colors)

    # Draw tiles

    # Coords to draw to
    coords = []
    for i in range(width):
        for j in range(height):
            coords.append([i, j])
    # Optionally shuffle coords to avoid gradients
    if shuffle:
      random.shuffle(coords)

    # Run through finding the best tile for the position        
    for c in coords:
        i = c[0]
        j = c[1]

        pixel = resized_target_image.getpixel((i, j))  # Getthe pixel color at (i, j)
        closest = tree.query(pixel)             # Returns (distance, index)
        index = closest[1]        # We only need the index

        # Offset of tile
        x, y = i*tile_size[0], j*tile_size[1]

        # Draw tile
        output.paste(tiles[index], (x, y))

        # remove tile and recompute the tree
        if remove_when_used:
          colors.pop(index)
          tiles.pop(index)
          tree = spatial.KDTree(colors)

    # Save output
    output.save("mosaic.jpg")

In [ ]:
make_mosaic(dataset,required_num_images)
Image.open('mosaic.jpg').resize((width*16, height*16))#.save('./assets/mosaic_lowres.jpg')